In [1]:
# IMPORTS
# Native
import os
import pickle
from datetime import date

# 3th party
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from progressbar import progressbar
import os.path as osp
import mmcv

In [2]:
# General setting
today = date.today()
today = today.strftime('%Y_%m_%d')


# Directory settings
origin_dir = "../data/tracked_fish_3_camera_angles"
destination_dir = f"../data/{today}_coco_select_vids_data_change"
video_dir = "tracking_vids"
label_dir = "h5_tracking_files"
camera_angles = ["A","B","C"]

# Predator = 0
# Prey = 1
#classes = ["0", "1"]

# annotation setting
radius = 50

selected_vids = ["131", "144", "148", "160", "204", "238", "242", "243", "252", "283", "297", "306", "325", "343", "373", "400", "413", "417", "419", "447", "449", "455", "456", "463"]

In [3]:
vid_list = sorted(os.listdir(origin_dir))
vid_list = selected_vids

print(vid_list)
train, test = train_test_split(vid_list, test_size=0.2, random_state=1)
train, val = train_test_split(train, test_size=0.2, random_state=1)

print(train)
print(val)
print(test)


['131', '144', '148', '160', '204', '238', '242', '243', '252', '283', '297', '306', '325', '343', '373', '400', '413', '417', '419', '447', '449', '455', '456', '463']
['204', '283', '447', '325', '455', '297', '456', '417', '238', '306', '413', '144', '463', '400', '242']
['148', '252', '243', '131']
['343', '419', '160', '373', '449']


In [4]:
# Loop over all subdir
def convert_fish_tracking_to_coco(split, data_dir, destination_dir, out_file):
    dest_img_path = osp.join(destination_dir, "images")
    dest_label_path = osp.join(destination_dir, "labels")
    out_file = osp.join(destination_dir, out_file)
    # Create directories if not exists
    if not os.path.exists(destination_dir):
        os.makedirs(destination_dir)
        os.makedirs(dest_img_path)
        os.makedirs(dest_label_path)


    annotations = []
    images = []
    obj_count = 0
    # osp.join(data_dir, split)
    for d in progressbar(sorted(split)):
        # print(d)
        # print(data_dir)
        sub_dir = osp.join(data_dir, d)
        video_path = osp.join(sub_dir, video_dir)
        annotation_path = osp.join(sub_dir, label_dir)
    
        for angle in camera_angles:
            annotation_file = osp.join(annotation_path, f"{d}Corr{angle}.pkl")
            annotation_dict = pickle.load(open(annotation_file, "rb"))

            # Load video
            video_file = os.path.join(video_path, f"{d}_{angle}.mp4")
            cap = cv2.VideoCapture(video_file)

            i = 0
            for x,y in zip(annotation_dict["0"]["X"],annotation_dict["0"]["Y"]):
            
                ret, frame = cap.read()

                if np.isnan(x) or np.isnan(y):
                    continue
                
                x_min = int(x) - radius
                y_min = int(y) - radius
                x_max = int(x) + radius
                y_max = int(y) + radius

                # Get frame size
                width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH ))
                height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT ))
                # fps =  cap.get(cv2.CAP_PROP_FPS)

                image = frame

                # Naming scheme
                idx = str(i)
                idx = idx.zfill(5) 
                image_name = f"{d}_{angle}_{idx}"
                
                # Write img
                cv2.imwrite(f"{dest_img_path}/{image_name}.jpg", image)
                
                images.append(dict(
                    id=obj_count,
                    file_name=f"{image_name}.jpg",
                    height=height,
                    width=width))
                # write label
                
                label = ["predator", "0", x_min, y_min, x_max, y_max, width, height]
                with open(f'{dest_label_path}/{image_name}.txt', 'w') as f:
                    for item in label:
                        f.write("%s " % item)
                
                data_anno = dict(
                    image_id=obj_count,
                    id=obj_count,
                    category_id=0,
                    bbox=[x_min, y_min, x_max - x_min, y_max - y_min],
                    area=(x_max - x_min) * (y_max - y_min),
                    # segmentation=[],
                    iscrowd=0)
                annotations.append(data_anno)
                
                i+= 1
                obj_count+= 1 
    
    coco_format_json = dict(
        images=images,
        annotations=annotations,
        categories=[{'id':0, 'name': 'predator'}])
    mmcv.dump(coco_format_json, out_file)

In [5]:
convert_fish_tracking_to_coco(train, origin_dir, destination_dir, "train_coco.json")
convert_fish_tracking_to_coco(test, origin_dir, destination_dir, "test_coco.json")
convert_fish_tracking_to_coco(val, origin_dir, destination_dir, "val_coco.json")

100% (15 of 15) |########################| Elapsed Time: 0:04:01 Time:  0:04:01
100% (5 of 5) |##########################| Elapsed Time: 0:01:03 Time:  0:01:03
100% (4 of 4) |##########################| Elapsed Time: 0:00:54 Time:  0:00:54
